# Deep Research
>A Deep Research agent is broadly applicable to any business area, and to your own day-to-day activities. You can make use of this yourself

In [1]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [2]:
load_dotenv(override=True)

True

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool. That can add up to $2-$3 for the next 2 labs. We'll use low cost Search tools with other platforms, so feel free to skip running this if the cost is a concern.

Costs are here: https://platform.openai.com/docs/pricing#web-search

"Instructions" from OpenAI.          
tool=[WebSearchTool(search_context_size="low"))], search_context_size="low" is optional parameter. Search context size. 'Low' is cheaper. 'medium', 'high'         
model_settings=ModelSettings(tool_choice="required") Agent is required to run the tool.      

In [3]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [4]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

As of May 2025, several AI agent frameworks have emerged, each offering unique capabilities for developing intelligent, autonomous systems. Notable among these are LangChain, AutoGen, CrewAI, LangGraph, and AutoAgent.

**LangChain** is a versatile framework that enables developers to build applications by chaining together prompts, models, memory, and external tools. It supports multiple language models and integrates with various tools, facilitating the creation of complex workflows. Its modular design and extensive ecosystem make it a popular choice for developing custom AI applications. ([medium.com](https://medium.com/%40elisowski/top-ai-agent-frameworks-in-2025-9bcedab2e239?utm_source=openai))

**AutoGen**, developed by Microsoft, focuses on multi-agent systems and code automation. It allows agents to communicate in natural language, coordinating tasks such as planning, development, and review. This conversational approach simplifies the creation of collaborative AI systems, enhancing productivity and efficiency. ([medium.com](https://medium.com/%40elisowski/top-ai-agent-frameworks-in-2025-9bcedab2e239?utm_source=openai))

**CrewAI** emphasizes multi-agent collaboration, enabling the creation of teams of AI agents with specialized roles. This framework is designed for complex workflows requiring coordination and task delegation, making it suitable for applications in logistics, resource planning, and collaborative problem-solving. ([medium.com](https://medium.com/%40elisowski/top-ai-agent-frameworks-in-2025-9bcedab2e239?utm_source=openai))

**LangGraph** extends LangChain by providing a graph-based structure for managing agent workflows. It supports complex stateful systems with advanced logic, including loops and conditional branching, and integrates with LangChain’s tools and models. This framework is ideal for applications requiring precise, sequential task execution, such as healthcare or supply chain management. ([lekha-bhan88.medium.com](https://lekha-bhan88.medium.com/top-5-agentic-ai-frameworks-to-watch-in-2025-9d51b2b652c0?utm_source=openai))

**AutoAgent** offers a fully automated, zero-code framework for developing LLM agents. It enables users to create and deploy AI agents through natural language, making AI development accessible to those without technical expertise. Operating as an autonomous agent operating system, AutoAgent comprises components like agentic system utilities, an LLM-powered actionable engine, and a self-managing file system. ([arxiv.org](https://arxiv.org/abs/2502.05957?utm_source=openai))

These frameworks represent the forefront of AI agent development, each contributing to the evolution of intelligent, autonomous systems across various domains. 

**LangChain**: Facilitates building applications powered by large language models (LLMs), simplifying complex workflows like context management and multi-step tasks. (<a href="https://www.curotec.com/">curotec.com</a>)       
**LangGraph**: Expands on LangChain, focusing on multi-agent systems that can collaborate and adapt, featuring coordination tools and visual graph-based workflows. (<a href="https://www.curotec.com/">curotec.com</a>)        
**CrewAI**: Designed for teamwork, its role-based architecture helps specialized agents work together effectively, offering dynamic task planning and conflict resolution. (<a href="https://www.curotec.com/">curotec.com</a>)         
**Microsoft Semantic Kernel**: Integrates traditional development tools with AI capabilities, supporting multiple languages and emphasizing security and compliance. (<a href="https://www.curotec.com/">curotec.com</a>)       
**Microsoft AutoGen**: Built for advanced multi-agent systems, offering flexibility and enhanced context management, with a modular architecture and human-in-the-loop capabilities. (<a href="https://www.curotec.com/">curotec.com</a>)        
**OpenAI Gym**: Widely used for reinforcement learning, offering diverse training environments for agents, from games to robotics. (<a href="https://www.curotec.com/">curotec.com</a>)       
**Rasa**: An open-source framework for developing customized conversational AI solutions, highly customizable and integrating with various platforms.  (<a href="https://www.curotec.com/">curotec.com</a>)        
**Jade (Java Agent Development Framework)**: A lightweight framework tailored for distributed systems, especially in IoT environments, with strong communication protocols for distributed systems.  (<a href="https://www.curotec.com/">curotec.com</a>)       

### As always, take a look at the trace

https://platform.openai.com/traces

### We will now use Structured Outputs, and include a description of the fields  
we're going to make heavier use of structured outputs than previously       
planner_agent, is responsible for taking a query and coming up with a handful of searches, that it should run based on that query in order to do some deep research        
No of search is set to 3. Bigger number will result in a better result but also increase the cost.      
     
Asking for a reason, something similar to reasoning style behavior, chain of thought prompting.      
     
WebSearchPlan, is a list of WebSearchItems

In [5]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."


class WebSearchItem(BaseModel):
    reason: str
    "Your reasoning for why this search is important to the query."

    query: str
    "The search term to use for the web search."


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]
    """A list of web searches to perform to best answer the query."""


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [6]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find recent developments and emerging frameworks in AI agents for 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To explore comparisons and features of popular AI agent frameworks in 2025.', query='best AI frameworks for agents 2025'), WebSearchItem(reason='To gather information on industry trends and predictions for AI agents in 2025.', query='AI agents technology trends 2025')]


What we get back is an object which has a field searches and field list is WebSearchItems with reason and query.     

### Send Email

decorator, that will convert a function into a tool. 

In [7]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("rsolo@ranaa.ca") # Change this to your verified email
    to_email = To("ranjitsolo@yahoo.com") # Change this to your email
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [8]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given subject and HTML body', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000023559DA6D40>, strict_json_schema=True)

email agent

In [9]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)

researcher agent    
Pydantic object structured output

In [10]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str
    """A short 2-3 sentence summary of the findings."""

    markdown_report: str
    """The final report"""

    follow_up_questions: list[str]
    """Suggested topics to research further"""


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

### The next 3 functions will plan and execute the search, using planner_agent and search_agent   
First 3 out of 5 functions.     
Execute a search using planner_agent and search_agent    
      
plan_searches, calls runner.run for passing planner_agent passing the query. This will comeback with number of searches. final output is returned.                 
      
perform_searches, actually going to do the searches for each. we're going to create a bunch of asyncio tasks to search for each item (search_plan is a pydantic object)    
then use gather approach, tasks will run in parallel and assigned to results.     
      
search agent, reason and the query, run the search agent and get the results.

In [11]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

### The next 2 functions write a report and email it

two housekeeping functions, one of them to write the report and other to send email

In [12]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

### Showtime!

In [13]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("Hooray!")

Starting research...
Planning searches...
Will perform 3 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Writing email...
Email sent
Hooray!
